In [8]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET

In [8]:
import requests
import pandas as pd
import json

url = 'http://apis.data.go.kr/5690000/sjTrafficLightInforamtion/sj_00000610'
params = {
    'serviceKey': "jLvCDpgyRWwv286cZZNgYlNM5QlnxkY/cqth3U6NFKM0me56sIRmA6IUYXaXVbBX/vsba5X+4SzuLE0PANbRcw==",
    'pageIndex': '1',
    'pageUnit': '3000',
    'dataTy': 'json',
}

response = requests.get(url, params=params)

# 응답에서 body > items 만 추출
response_json = response.json()
items = response_json.get('body', {}).get('items', [])

# 필요한 컬럼만 필터링
columns_needed = [
    'roadRouteNm', 'la', 'lo', 'sgngnrInstlMthdSe', 'tfclghtSe',
    'tfclghtOrdr', 'tfclghtHr', 'signlHrDcsnMthdSe', 'mngInstNm',
    'telno', 'flasherOperBgngTm', 'flasherOperEndTm'
]

# DataFrame 생성 및 컬럼 선택
df = pd.DataFrame(items)
df = df[columns_needed]

# 결과 저장
df.to_csv('./dataset/traffic_lights_filtered.csv', index=False, encoding='utf-8-sig')

print("CSV 파일 생성 완료: traffic_lights_filtered.csv")


CSV 파일 생성 완료: traffic_lights_filtered.csv


In [1]:
!pip install simplekml

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for simplekml: filename=simplekml-1.3.6-py3-none-any.whl size=65953 sha256=515ae8c5722d5710a788d73ceb353523511b72d0a1fccbe366e250afd2794615
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\fd\2a\d1\84a7abdbb59566c442beb1b4fb946fe1e26c5a48fd1d5e0763
Successfully built simplekml


In [7]:
import csv
import simplekml

csv_file_path = './dataset/traffic_lights_filtered.csv'
kml_file_path = './dataset/sejong_traffic_lights.kml'

kml = simplekml.Kml()

with open(csv_file_path, newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    reader.fieldnames = [name.strip() for name in reader.fieldnames]

    for row in reader:
        row = {k.strip(): v.strip() for k, v in row.items()}

        try:
            latitude = float(row['la'])
            longitude = float(row['lo'])

            description = (
                f"도로명: {row['roadRouteNm']}\n"
                f"설치방식: {row['sgngnrInstlMthdSe']}\n"
                f"신호등 종류: {row['tfclghtSe']}\n"
                f"신호 순서: {row['tfclghtOrdr']}\n"
                f"신호등 운영시간: {row['tfclghtHr']}\n"
                f"운영시간 결정 방식: {row['signlHrDcsnMthdSe']}\n"
                f"관리기관: {row['mngInstNm']}\n"
                f"연락처: {row['telno']}\n"
                f"점멸기 시작시간: {row['flasherOperBgngTm']}\n"
                f"점멸기 종료시간: {row['flasherOperEndTm']}"
            )

            kml.newpoint(
                name=row['roadRouteNm'],
                coords=[(longitude, latitude)],
                description=description
            )
        except Exception as e:
            print(f"오류 발생 - 행 건너뜀: {e}")

kml.save(kml_file_path)
print(f"KML 파일이 생성되었습니다: {kml_file_path}")


KML 파일이 생성되었습니다: ./dataset/sejong_traffic_lights.kml


In [5]:
import pandas as pd
df = pd.read_csv("./dataset/traffic_lights_filtered.csv")

In [6]:
df.columns

Index(['roadRouteNm', 'la', 'lo', 'sgngnrInstlMthdSe', 'tfclghtSe',
       'tfclghtOrdr', 'tfclghtHr', 'signlHrDcsnMthdSe', 'mngInstNm', 'telno',
       'flasherOperBgngTm', 'flasherOperEndTm'],
      dtype='object')

In [6]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# API 기본 정보
url = 'http://apis.data.go.kr/5690000/sjCrosswalkInformation/sj_00001170'
params = {
    'serviceKey': 'jLvCDpgyRWwv286cZZNgYlNM5QlnxkY/cqth3U6NFKM0me56sIRmA6IUYXaXVbBX/vsba5X+4SzuLE0PANbRcw==',
    'pageUnit': '1000',
    'dataTy': 'xml',
    'searchCondition': 'cartrk_Co',
    'searchKeyword': '4'
}

all_items = []
page = 1

while True:
    params['pageIndex'] = str(page)
    response = requests.get(url, params=params)
    xml_text = response.content.decode('utf-8')
    root = ET.fromstring(xml_text)

    current_items = root.findall('.//item')
    if not current_items:
        break

    for item in current_items:
        row = {
            '도로명': item.findtext('roadNm'),
            '도로명주소': item.findtext('roadNmAddr'),
            '지번주소': item.findtext('addr'),
            '녹색신호시간': item.findtext('greenSignlHr'),
            '적색신호시간': item.findtext('redSignlHr'),
            '위도': item.findtext('la'),
            '경도': item.findtext('lo'),
        }
        all_items.append(row)

    page += 1

# DataFrame 변환 및 저장
df = pd.DataFrame(all_items)
df.to_csv('./dataset/crosswalk_info.csv', index=False, encoding='utf-8-sig')

print(f"{len(df)}개의 항목을 수집하여 CSV로 저장했습니다.")


5개의 항목을 수집하여 CSV로 저장했습니다.
